## Design patterns with first-class functions

![Alt](class_strategy.JPG)

>Context
- Provides a service by delegating some computation to interchangeable components that implement alternative algorithms. In the e-commerce example, the context is an Order, which is configured to apply a promotional discount according to one of several algorithms.

>Strategy
- The interface common to the components that implement the different algorithms.In our example, this role is played by an abstract class called Promotion.

>Concrete Strategy
- One of the concrete subclasses of Strategy. FidelityPromo, BulkPromo and Large OrderPromo are the three concrete strategies implemented.

In [121]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity

In [122]:
class Order: # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
    
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

In [123]:
class Promotion(ABC): # the Strategy: an Abstract Base Class
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""

        
class FidelityPromo(Promotion): # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0

    
class BulkItemPromo(Promotion): # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount

    
class LargeOrderPromo(Promotion): # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0

##### Example 6-2. Sample usage of Order class with different promotions applied.

In [124]:
# Two customers: joe has 0 fidelity points, ann has 1100.
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

# One shopping cart with three line items
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

# The FidelityPromo promotion gives no discount to joe
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [125]:
# ann gets a 5% discount because she has at least 1000 points.
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [126]:
# The banana_cart has 30 units of the "banana" product and 10 apples
banana_cart = [LineItem('banana', 30, .5),LineItem('apple', 10, 1.5)]

# Thanks to the BulkItemPromo, joe gets a $1.50 discount on the bananas.
Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [127]:
# long_order has 10 different items at $1.00 each
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]

# joe gets a 7% discount on the whole order because of LargerOrderPromo
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [128]:
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [129]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

## Function-oriented Strategy

##### Example 6-3. Order class with discount strategies implemented as functions.

In [130]:
from collections import namedtuple
Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
    
    def total(self):
        return self.price * self.quantity
    

    
class Order: # the Context
    
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    # To compute a discount, just call the self.promotion() function.
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())    

    # No abstract class.
    # Each strategy is a function.
def fidelity_promo(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


def large_order_promo(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

In [131]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)

cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]

In [132]:
# To apply a discount strategy to an Order, just pass the promotion function as an argument.
Order(joe, cart, fidelity_promo)

<Order total: 42.00 due: 42.00>

In [133]:
Order(ann, cart, fidelity_promo)

<Order total: 42.00 due: 39.90>

In [134]:
banana_cart = [LineItem('banana', 30, .5),LineItem('apple', 10, 1.5)]
Order(joe, banana_cart, bulk_item_promo)

<Order total: 30.00 due: 28.50>

In [135]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, large_order_promo)

<Order total: 10.00 due: 9.30>

In [136]:
Order(joe, cart, large_order_promo)

<Order total: 42.00 due: 42.00>

## Choosing the best strategy: simple approach

##### Example 6-6. best_promo finds the maximum discount iterating over a list of functions.

In [137]:
# promos: list of the strategies implemented as functions.
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

# best_promo takes an instance of Order as argument, as do the other *_promo functions.
def best_promo(order):
    """Select best discount available """
    return max(promo(order) for promo in promos)

In [138]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [139]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [140]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

## Finding strategies in a module

Modules in Python are also first class objects, and the Standard Library provides several
functions to handle them. The built-in globals is described as follows in the Python
docs:

- globals():Return a dictionary representing the current global symbol table. This is always the dictionary of the current module (inside a function or method, this is the module where it is defined, not the module from which it is called).

In [141]:
# Iterate over each name in the dictionary returned by globals().
# Select only names that end with the _promo suffix
# Filter out best_promo itself, to avoid an infinite recursion
promos = [globals()[name] for name in globals() if name.endswith('_promo') and name != 'best_promo']
promos

[<function __main__.fidelity_promo(order)>,
 <function __main__.bulk_item_promo(order)>,
 <function __main__.large_order_promo(order)>]

In [142]:
# No changes inside best_promo
def best_promo(order):
    """ Select best discount available """
    return max(p(order) for p in promo)

##### Example 6-8. The promos list is built by introspection of a new promotions module.

In [272]:
import sys
import os
sys.path.append('promotions')

In [281]:
import inspect
import promotions

In [282]:
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
promos 

[]

In [262]:
def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)